# News Research Agent

In [1]:
import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool, ScrapeWebsiteTool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import json

# Libaries for sending email
import base64
import pickle
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

## Create Agents, Tasks and Crew

In [2]:
load_dotenv()
serper_api_key = os.getenv("SERPER_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [3]:
search_tool = SerperDevTool(
    search_url = "https://google.serper.dev/news",
    n_results = 10
)
website_tool = WebsiteSearchTool()
scrape_tool = ScrapeWebsiteTool()

In [4]:
topic = input("Enter the research topic: ")

In [5]:
from datetime import datetime

# Get the current date and time
now = datetime.now()

# Format the date and time
formatted_date = now.strftime("%Y-%m-%d %H:%M:%S")

In [6]:
# file name

file_name_txt = f"""report_{topic}_{formatted_date}.txt"""
file_name_html = f"""report_{topic}_{formatted_date}.html"""


In [7]:
trusted_sources = ['official government websites (e.g. ending on .gov)','BBC', 'CNN', 'The Guardian', 'The New York Times', 'The Washington Post', 'The Wall Street Journal', 'Neue Zürcher Zeitung', ]

In [8]:
researcher = Agent(
    role = "Senior News Research Specialist",
    goal = "Conduct comprehensive research and gather detailed information to support C-Level Executives in their decision making.",
    backstory = """Expert researcher skilled at discovering the latest trends on a given issue. One of your main skills is finding the right search terms to find the most relevant papers on a given topic. Specializes in throrough and detailed research""",
    verbose = False,
    allow_delegation = False,
    tools = [search_tool],
    llm = ChatOpenAI(model_name="gpt-4o", temperature=0.7)
    #max_iter = 10
)

analyst = Agent(
    role = "Senior News Reporting Analyst",
    goal = "Analyze and synthesize research findings",
    backstory = """Expert analyst skilled at processing complex information and identifying key patterns and insights. Spezialises in clear and actionable analysis.""",
    verbose = False,
    allow_delegation = False,
    tools = [search_tool],
    #tools = [scrape_tool],
    llm = ChatOpenAI(model_name="gpt-4o", temperature=0.7)
    #max_iter = 10
)

writer = Agent(
    role = "Content synthesis expert",
    goal = "Create clear, structured reports from analysis",
    backstory = """You are a well-respected content strategist with a knack for creating engaging and informative articles.
    Your expertise lies in transforming complex concepts into clear, compelling narratives that are easily understood by a broad audience.
    Your main focus is on informing C-Level Executives about the latest trends and developments of the given topic""",
    verbose = True,
    allow_delegation = False,
    llm = ChatOpenAI(model_name="gpt-4o", temperature=0.7)
    #max_iter = 10
)

email_agent = Agent(
    role="Email Formatter",
    goal="Generate well-formatted HTML emails for professional communication.",
    backstory="You are an expert email composer who specializes in crafting visually appealing, structured HTML emails that look great in Gmail and other email clients.",
    verbose=True,
    allow_delegation=False,
    llm = ChatOpenAI(model_name="gpt-4o", temperature=0.7)
)

In [9]:
task1 = Task(
    description = f"""Research the latest developments, news and facts about the topic of {topic} as of the date {formatted_date}:
    
    Follow these steps:
    1. Find reliable sources and the latest information on the topic. Focus on trusted sources such as {trusted_sources}.
    2. Extract key details and evidence
    3. Verify the information across sources
    4. Document your findings with references and exact URLs""",
    expected_output = "Detailed list of research findings with title, date and source url.",
    agent = researcher
)

task2 = Task(
    description = f"""Analyze the latest developments, news and facts about {topic}:
    
    Follow these steps:
    1. Review and categorize the information
    2. Identify patterns and trends
    3. Evaluate credibility and relevance
    4. Note key insights""",
    expected_output = "Analysis of findings and key insights",
    agent = analyst,
    context = [task1]
)

task3 = Task(
    description = f"""Create a clear report on the latest developments, news and facts about {topic}:
    
    Include:
    1. Brief executive summary with the latest developments. Focus on always mentioning from when (Day, Date) the information is.
    2. A bulleted list of key insights.
    3. At the end and only at the end list the URLs of the sources and the publishing date of the sources.""",
    expected_output = "Structured report with key insights. ALWAYS list the URLs of the sources and the publishing date of the sources at the end of the report.",
    agent = writer,
    context = [task1, task2],
    output_file= file_name_txt
)

task4 = Task(
    description = f"""Your task is to generate a **well-formatted HTML email** based on the following user input.

    - Ensure that the email includes proper **HTML structure**.
    - Use `<h2>` for headings, `<p>` for paragraphs, and `<b>` or `<strong>` for bold text.
    - If a user mentions links, use `<a href="URL">Link Text</a>`.
    - Ensure compatibility with Gmail by avoiding heavy CSS.
    Output a **valid HTML email** inside <html> and <body> tags.""",
    expected_output = "A transformed text from normal text to HTML",
    agent = email_agent,
    context = [task3],
    output_file= file_name_html
)

In [10]:
crew = Crew(
    agents = [researcher, analyst, writer, email_agent],
    tasks = [task1, task2, task3, task4],
    verbose = 1
)

In [ ]:
final = crew.kickoff()

In [12]:
# print(f"Raw Output: {final.raw}")
# final.raw

## Send Email

In [13]:
# Define the scope for Gmail API
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

def authenticate_gmail():
    """Authenticate and return the Gmail API service using pickle."""
    creds = None
    token_path = 'token.pickle'  # Using pickle for credential storage

    # Load saved credentials if they exist
    if os.path.exists(token_path):
        with open(token_path, 'rb') as token:
            creds = pickle.load(token)

    # If no valid credentials, go through OAuth flow
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0, open_browser=False)  # ✅ Fix here

        # Save credentials for future use
        with open(token_path, 'wb') as token:
            pickle.dump(creds, token)

    return build('gmail', 'v1', credentials=creds)


def create_message(sender, to, subject, html_body):
    """Create an email message."""
    message = MIMEMultipart()
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    # Attach the HTML body
    message.attach(MIMEText(html_body, "html"))
    raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode('utf-8')
    return {'raw': raw_message}

def send_email(service, sender, to, subject, body):
    """Send an email via Gmail API."""
    message = create_message(sender, to, subject, body)
    try:
        send_message = service.users().messages().send(userId="me", body=message).execute()
        print(f"Email sent successfully! Message ID: {send_message['id']}")
    except Exception as e:
        print(f"An error occurred: {e}")


In [14]:
gmail_service = authenticate_gmail()


In [15]:
# Read the HTML content from a file
html_file_path = file_name_html  # Update with your actual file path

with open(html_file_path, "r", encoding="utf-8") as file:
    body = file.read()  # Load the HTML content from the file

sender_email = "jkdevpy@gmail.com"  # Replace with your email
recipient_email = "j.piccolatorta@gmail.com"  # Replace with recipient email
subject = f"""Report on {topic} / {formatted_date}"""

send_email(gmail_service, sender_email, recipient_email, subject, body)



Email sent successfully! Message ID: 194f68f50183ace7
